# TODO
* [ ] Correrlo con `max_iter = 5_000`
* [ ] Hacer la versión Batch
* [ ] Hacer la versión proyectada

Notebook que sirve de ejemplo para calcular el baricentro de un conjunto de datos de imágenes.

# Configuraciones iniciales

## Constantes

In [1]:
NOTEBOOK = 1
CLEAN_LOGS = True  # If you want to clean the logs directory
SAVE_FIGS = True  # If you want to save the figures.

REPORT_EVERY = 100  # To report at the logger
MAX_ITER = 5_000  # MAx number of iterations for the SGDW

MAX_ITER = 50; REPORT_EVERY = 5  # Descomentar para debuguear

In [2]:
import random

import numpy as np
import torch

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.backends.cudnn.deterministic = True

In [3]:
from pathlib import Path
from icecream import ic


DS_NAME = "data"

CURR_PATH = Path().absolute()
ic(CURR_PATH)
BASE_PATH = CURR_PATH.parent.parent
ic(BASE_PATH)
DATA_PATH = BASE_PATH / "data"
ic(DATA_PATH)
WGAN_PATH = BASE_PATH / "wgan_gp"
ic(WGAN_PATH)
NETS_PATH = WGAN_PATH / "networks" 
ic(NETS_PATH)
IMGS_PATH = CURR_PATH / "imgs" / f"notebook-{NOTEBOOK:02d}"
IMGS_PATH.mkdir(parents=True, exist_ok=True)
ic(IMGS_PATH)

In [4]:
def save_fig(fig, name_to_save):
    if SAVE_FIGS:
        PATH_TO_SAVE = IMGS_PATH / name_to_save
        fig.savefig(PATH_TO_SAVE.with_suffix(".pdf"))
        fig.savefig(PATH_TO_SAVE.with_suffix(".png"))

## Importaciones generales

In [5]:
from icecream import ic
from bwb.sgdw import sgdw
from bwb.sgdw import wrappers
from bwb.distributions import *
from bwb.sgdw.plotters import PlotterComparison

## Configuraciones 

In [6]:
from bwb.config import conf

conf.use_gpu()
conf.use_single_precision()
conf.set_eps(1e-16)
conf

## Configuración del Logger

In [7]:
import time
from pathlib import Path


# Create the logs directory
LOG_PATH = Path("logs") / f"notebook-{NOTEBOOK:02d}_{time.strftime('%Y%m%d_%H%M%S')}.log"
if not LOG_PATH.parent.exists():
    LOG_PATH.parent.mkdir()

# Clean the logs
if CLEAN_LOGS:
    for log_file in Path("logs").glob(f"notebook-{NOTEBOOK:02d}*.log"):
        log_file.unlink()

In [8]:
import logging
from bwb.logging_ import log_config


# Remove the handlers
log_config.remove_all_handlers()
ic(log_config.loggers)

# Define and add FileHandler
fh = logging.FileHandler(LOG_PATH)
log_config.FORMATTER = fh
log_config.add_handler(fh)


_log = log_config.get_logger("notebook")
log_config.set_level(level=logging.DEBUG, name="notebook")
log_config.set_level(level=logging.DEBUG, name="bwb.sgdw.sgdw")
log_config.set_level(level=logging.DEBUG, name="bwb.sgdw.plotters")

Esta celda es para configurar la información mostrada en el logger

In [9]:
# Set the default options for the report
wrappers.ReportProxy.INCLUDE_OPTIONS = wrappers.ReportOptions(
    dt=False,
    dt_per_iter=True,
    iter=True,
    step_schd=True,
    total_time=True,
    w_dist=False
)

## Obtención del dataset

In [10]:
# You can use the wrapper to transform the usual DataSet into a model set
from bwb.distributions.models import ModelDataset
import quick_torch as qt
import torchvision.transforms.v2 as T

transform_ds = T.Compose([
    T.Resize((32, 32)),
    T.ToImage(),
    T.ToDtype(conf.dtype, scale=True),
    T.Lambda(lambda x: x.squeeze()),
])

def get_ds(file_path, transform=transform_ds):
    ic(file_path)
    categories = [qt.Category.FACE]
    dataset_ = qt.QuickDraw(
        root=DATA_PATH,
        categories=categories,
        transform=transform,
        download=True,
        recognized=True,
    )
    path_dataset = Path(file_path)
    dataset_.data = np.load(path_dataset).reshape(-1, 28, 28)
    dataset_.targets = np.ones(len(dataset_.data), dtype=int)
    dataset = dataset_.get_train_data()
    ic(len(dataset))

    return ModelDataset(dataset)

DS_PATH = WGAN_PATH / "dataset" / "cleaned" / f"{DS_NAME}.npy"
ds_models = get_ds(DS_PATH)

ds_models.get(0)

In [11]:
ds_models

In [16]:
models = [ds_models.get(i) for i in range(100)]

from bwb.distributions import stats

stats.plot_log_like_models(models, data)

## Obtener GAN

De la misma manera, se puede definir un muestreador de distribuciones utilizando una GAN. Para ello, empezamos definiendo las redes neuronales a utilizar

In [12]:
from wgan_gp.wgan_gp_vae.model_resnet import Generator, Encoder, LatentDistribution
import torch
from wgan_gp.wgan_gp_vae.utils import load_checkpoint


device = conf.device

NOISE = "norm"
LATENT_DIM = 128
CHANNELS_IMG = 1
NUM_FILTERS = [256, 128, 64, 32]

noise_sampler = LatentDistribution(NOISE, LATENT_DIM, device)


G = Generator(LATENT_DIM, CHANNELS_IMG, latent_distr=NOISE).to(device)
E = Encoder(LATENT_DIM, CHANNELS_IMG).to(device)

DS_NAME = "data"
FACE_PATH = NETS_PATH / f"cleaned_{DS_NAME}_zDim{LATENT_DIM}_{NOISE}_bs_128"
ic(FACE_PATH)

load_checkpoint(G, FACE_PATH, "generator", device)
load_checkpoint(E, FACE_PATH, "encoder", device)

G.eval(); E.eval()
print()

In [13]:
noise_sampler

In [14]:
from bwb.distributions import DistributionDraw
from torchvision import disable_beta_transforms_warning
disable_beta_transforms_warning()

import torchvision.transforms.v2 as T


z = noise_sampler(1)
m = G(z)

transform_in = T.Compose([
    T.Lambda(lambda x: x / torch.max(x)),
    T.ToPILImage(),
    T.Resize(32),
    T.ToImage(),
    T.ConvertImageDtype(conf.dtype),
    T.Normalize((0.5,), (0.5,)),
])

transform_out_ = T.Compose([
    T.ToDtype(conf.dtype),
    T.Lambda(lambda x: x.squeeze()),
    T.Lambda(lambda x: x - torch.min(x)),
    T.Lambda(lambda x: x / torch.sum(x)),
])

transform_out = T.Compose([
    transform_out_,
    T.Lambda(lambda x: DistributionDraw.from_grayscale_weights(x)),
])


out: DistributionDraw = transform_out(m)
print(out.dtype)
out

## Definir $\gamma_k$

Aquí se utiliza una función de la forma
\begin{equation*}
    \gamma_k = \frac{a}{(b^{1/c} + k)^c}
\end{equation*}

Con $a > 0$, $b \geq 0$ y $0.5 < c \leq 1$

La idea es que cuando $k=0$, $\gamma_0 = \frac{a}{b}$ es la proporción entre $a$ y $b$, permitiendo ajustar el valor inicial.

In [15]:
from bwb.sgdw.utils import step_scheduler
window = 5

def test_gamma(gamma):

    for t in range(window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 50
    for t in range(init, init+window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 100
    for t in range(init, init+window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 300
    for t in range(init, init+window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 500
    for t in range(init, init+window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 1_000
    for t in range(init, init+window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 3_000
    for t in range(init, init+window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 5_000
    for t in range(init, init+window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()


_a = 3
_eps = 1e-3
params = dict(a=_a, b=_a+1e-2, c=0.5+_eps)
# params = dict(a=1, b=1, c=1)

gamma = step_scheduler(**params)

test_gamma(step_scheduler(**params))

# Baricentro de imágenes

Para obtener el baricentro de un conjunto de imágenes, es necesario utilizar la clase `UniformDiscreteSampler` y fijarla con una clase que tenga los siguientes métodos:
* `get(i) -> Distribution` que retorne la distribución $i$-ésima.
* `__len__() -> int` que retorne el tamaño del dataset.

In [16]:
distr_sampler = UniformDiscreteSampler[DistributionDraw]().fit(models=ds_models)

Luego definimos el algoritmo a utilizar. En este caso, utilizaremos `DebiesedDistributionDrawSGDW` que realiza los transportes utilizando un método convolución debiesed. 

In [17]:
dist_draw_sgdw = sgdw.DebiesedDistributionDrawSGDW(
    distr_sampler=distr_sampler,
    step_scheduler=step_scheduler(**params),
    max_iter=MAX_ITER,
)
dist_draw_sgdw = wrappers.ReportProxy(dist_draw_sgdw, report_every=REPORT_EVERY)
dist_draw_sgdw

Definimos una clase para comparar las imágenes de las muestras con la iteración del algoritmo.

In [18]:
plotter_comp = plotter_comp_ds = PlotterComparison(
    dist_draw_sgdw,
    n_cols=12,
    n_rows=2,
    cmap="binary_r",
    plot_every=24
)
plotter_comp.sgdw

In [19]:
_log.info(f"Running SGD-Wasserstein with '{DS_NAME}' DS barycenter")
bar = plotter_comp.run()
plotter_comp.sgdw

Obtenemos una visualización de las primeras imágenes.

In [20]:
fig, _ = plotter_comp.plot(0)
save_fig(fig, "first-iters-DS")

Obtenemos una visualización de las últimas imágenes.

In [21]:
fig, _ = plotter_comp.plot()
save_fig(fig, "last-iters-DS")

# Baricentro de la GAN

## Definir el algoritmo

In [22]:
distr_sampler = GeneratorDistribSampler()
distr_sampler.fit(generator=G, noise_sampler=noise_sampler, transform_out=transform_out_)
distr_sampler.draw()

In [23]:
dist_draw_sgdw = sgdw.DebiesedDistributionDrawSGDW(
    distr_sampler=distr_sampler,
    step_scheduler=step_scheduler(**params),
    max_iter=MAX_ITER,
)
dist_draw_sgdw = wrappers.ReportProxy(dist_draw_sgdw, report_every=REPORT_EVERY)
dist_draw_sgdw

Definimos una clase para comparar las imágenes de las muestras con la iteración del algoritmo.

In [24]:
from bwb.sgdw.plotters import PlotterComparison

plotter_comp = plotter_comp_gan = PlotterComparison(
    dist_draw_sgdw,
    n_cols=12,
    n_rows=2,
    cmap="binary_r",
    plot_every=24
)
plotter_comp.sgdw

In [25]:
_log.info(f"Running SGD-Wasserstein with '{DS_NAME}' GAN barycenter")
bar = plotter_comp.run()
dist_draw_sgdw.iter_params

Obtenemos una visualización de las primeras imágenes.

In [26]:
fig, _ = plotter_comp.plot(0)
save_fig(fig, "first-iters-GAN")

Obtenemos una visualización de las últimas imágenes.

In [27]:
fig, _ = plotter_comp.plot()
save_fig(fig, "last-iters-GAN")